In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as st
import scipy.optimize as opt
import matplotlib.pyplot as plt
from cvxopt import matrix, solvers
from datetime import datetime 

In [ ]:
from CAL.PyCAL import font

In [ ]:
def get_covmat(tickers, date, periods):
    '''
    输入tickers + 日期 + 过去天数，获得以此计算出来的年化协方差矩阵
    '''
    start_date = shift_date(date, periods)
    return_mat=DataAPI.MktEqudAdjGet(ticker=tickers, beginDate=start_date, endDate=date, field=u"ticker,tradeDate,closePrice",
                                     pandas="1")
    # 想想应该是fillna(0)还是dropna()
    return_mat = return_mat.pivot(index='tradeDate',columns='ticker', values='closePrice').pct_change().fillna(0.0)
    return return_mat.cov()*250


def get_smart_weight(cov_mat, method='min variance', wts_adjusted=False):
    '''
    功能：输入协方差矩阵，得到不同优化方法下的权重配置，
    
    输入：
        cov_mat  pd.DataFrame,协方差矩阵，index和column均为资产名称
        method  优化方法，可选的有min variance、risk parity、max diversification、equal weight
    输出：
        pd.Series  index为资产名，values为weight
    PS:
        依赖scipy package
    '''
    ## 什么玩意？为啥需要是dataframe
    if not isinstance(cov_mat, pd.DataFrame):
        raise ValueError('cov_mat should be pandas DataFrame！')
        
    omega = np.matrix(cov_mat.values)  # 协方差矩阵
    
    # 定义目标函数
    def fun1(x):
        return np.matrix(x) * omega * np.matrix(x).T
    
    def fun2(x):
        tmp = (omega * np.matrix(x).T).A1
        risk = x * tmp
        delta_risk = [sum((i - risk)**2) for i in risk]
        return sum(delta_risk)
    
    def fun3(x):
        den = x * omega.diagonal().T
        num = np.sqrt(np.matrix(x) * omega * np.matrix(x).T)
        return num/den
    
    # 初始值 + 约束条件 
    x0 = np.ones(omega.shape[0]) / omega.shape[0]  
    bnds = tuple((0,None) for x in x0)
    cons = ({'type':'eq', 'fun': lambda x: sum(x) - 1})
    options={'disp':False, 'maxiter':1000, 'ftol':1e-20}
        
    if method == 'min variance':   
        res = opt.minimize(fun1, x0, bounds=bnds, constraints=cons, method='SLSQP', options=options) 
    elif method == 'risk parity':
        res = opt.minimize(fun2, x0, bounds=bnds, constraints=cons, method='SLSQP', options=options)
    elif method == 'max diversification':
        res = opt.minimize(fun3, x0, bounds=bnds, constraints=cons, method='SLSQP', options=options)
    elif method == 'equal weight':
        return pd.Series(index=cov_mat.index, data=1.0 / cov_mat.shape[0])
    else:
        raise ValueError('method should be min variance/risk parity/max diversification/equal weight！！！')
        
    # 权重调整
    if res['success'] == False:
        # print res['message']
        pass
    wts = pd.Series(index=cov_mat.index, data=res['x'])
    if wts_adjusted == True:
        wts = wts[wts >= 0.0001]
        return wts / wts.sum() * 1.0
    elif wts_adjusted == False:
        return wts
    else:
        raise ValueError('wts_adjusted should be True/False！')
        
        
def get_idx_cons(idx, date):
    '''
    功能：获取指数在某一天的成分股列表
    输入：
        idx 指数，xxxxxx型string，000300沪深300，000016上证50，000905中证500，000906中证800，000001上证综指
              可以为多个指数组合，写法为['xxxxxx','xxxxxx']，此时返回的结果已经去除重复的ticker了！！！
        date yyyymmdd型string
    输出：
        list of tickers
    依赖：
        DataAPI：IdxConsGet
    '''
    
    try:
        data = DataAPI.IdxConsGet(ticker=idx,intoDate=date,field='',pandas="1")['consTickerSymbol']
    except:
        raise ValueError('DataAPI.IdxConsGet又出错了！！！')

    return list(set(data))

def get_dates(start_date, end_date, frequency='daily'):
    '''
    功能：输入起始日期和频率，即可获得日期列表（daily包括起始日，其余的都是位于起始日中间的）
    输入参数：
       start_date，开始日期，'xxxxxxxx'形式
       end_date，截止日期，'xxxxxxxx'形式
       frequency，频率，daily为所有交易日，daily1为所有自然日，weekly为每周最后一个交易日，weekly2为每隔两周，monthly为每月最后一个交易日，quarterly为每季最后一个交易日
    输出参数：
       获得list型日期列表，以'xxxxxxxx'形式存储
    PS:
        要用到DataAPI.TradeCalGet！！！
    '''
    
    data = DataAPI.TradeCalGet(exchangeCD=u"XSHG",beginDate=start_date,endDate=end_date,
                               field=u"calendarDate,isOpen,isWeekEnd,isMonthEnd,isQuarterEnd",pandas="1")
    if frequency == 'daily':
        data = data[data['isOpen'] == 1]
    elif frequency == 'daily1':
        pass
    elif frequency == 'weekly':
        data = data[data['isWeekEnd'] == 1]
    elif frequency == 'weekly2':
        data = data[data['isWeekEnd'] == 1]
        data = data[0:data.shape[0]:2]
    elif frequency == 'monthly':
        data = data[data['isMonthEnd'] == 1]
    elif frequency == 'quarterly':
        data = data[data['isQuarterEnd'] == 1]
    else:
        raise ValueError('调仓频率必须为daily/daily1/weekly/weekly2/monthly/quarterly！！！')
    # date_list = map(lambda x: x[0:4]+x[5:7]+x[8:10], data['calendarDate'].values.tolist())
    date_list = data['calendarDate'].values.tolist()
    return date_list


def shift_date(date, n, direction='back'): 
    '''
    功能：给定date，获取该日期前/后n个交易日对应的交易日
    输入：
        date  'yyyymmdd'类型字符串
        n  非负整数，取值区间（0,720）
        direction  方向，取值为back/forward
    PS：
        get_dates()
    '''
    
    last_two_year = str(int(date[:4])-3) + '0101'
    forward_two_year = str(int(date[:4])+3) + '1231'
    if direction == 'back':
        date_list = get_dates(last_two_year, date, 'daily')
        return date_list[len(date_list)-1-n]
    elif direction == 'forward':
        date_list = get_dates(date, forward_two_year, 'daily')
        return date_list[n]
    else:
        raise ValueError('direction should be back/forward！！！')


def ticker2sec(ticker):
    '''
    功能：将ticker转换为secID，输入的ticker不要有重复值！
    输入：
        tickers：list、dict、series，list时没有value，dict/series时有对应的value
    输出：
        同输入类型
    依赖：
        需要用到DataAPI.EquGet
    细节：
        若没找到这个sec，则返回结果不包含这个ticker
    '''
    
    universe = DataAPI.EquGet(equTypeCD=u"A",listStatusCD="L,S,DE,UN",field=u"ticker,secID",pandas="1") # 获取所有的A股（包括已退市）
    universe = dict(universe.set_index('ticker')['secID'])
    if isinstance(ticker, list):
        res = []
        for i in ticker:
            if i in universe:
                res.append(universe[i])
            else:
                print i, ' 在universe中不存在，没有找到对应的secID！'
        return res
    elif isinstance(ticker, dict):
        res = {}
        for i in ticker:
            if i in universe:
                res[universe[i]] = ticker[i]
            else:
                print i, ' 在universe中不存在，没有找到对应的secID！'
        return res
    elif isinstance(ticker, pd.Series):
        res = {}
        for i in ticker.index:
            if i in universe:
                res[universe[i]] = ticker[i]
            else:
                print i, ' 在universe中不存在，没有找到对应的secID！'
        return pd.Series(res)
    else:
        raise ValueError('ticker should be list or dict or series！')
        
        
def sec2ticker(sec):
    '''
    功能：将sec转换为ticker，输入的sec不要有重复值！
    输入：
        sec：list、dict、series，list时没有value，dict/series时有对应的value
    输出：
        同输入类型
    依赖：
        需要用到DataAPI：EquGet
    细节：
        若没找到对应的ticker，则返回结果不包含这个sec
    '''
    
    universe = DataAPI.EquGet(equTypeCD=u"A",listStatusCD="L,S,DE,UN",field=u"ticker,secID",pandas="1") # 获取所有的A股（包括已退市）
    universe = universe[['secID','ticker']]
    universe = dict(universe.set_index('secID')['ticker'])
    if isinstance(sec, list):
        res = []
        for i in sec:
            if i in universe:
                res.append(universe[i])
            else:
                print i, ' 在universe中不存在，没有找到对应的ticker！'
        return res
    elif isinstance(sec, dict):
        res = {}
        for i in sec:
            if i in universe:
                res[universe[i]] = sec[i]
            else:
                print i, ' 在universe中不存在，没有找到对应的ticker！'
        return res
    elif isinstance(sec, pd.Series):
        res = {}
        for i in sec.index:
            if i in universe:
                res[universe[i]] = sec[i]
            else:
                print i, ' 在universe中不存在，没有找到对应的ticker！'
        return pd.Series(res)
    else:
        raise ValueError('sec should be list or dict or series！')
        
        
def cal_maxdrawdown(data):
    '''
    功能：给定净值数据（list, np.array, pd.Series, pd.DataFrame），返回最大回撤
    输入：
        data, list/np.array/pd.Series/pd.DataFrame，净值曲线，初始金为1
    输出：
        list/np.array/pd.Series返回float
        pd.DataFrame返回pd.DataFrame，index为DataFrame.columns
    '''
    
    if isinstance(data, list):
        data = np.array(data)
    if isinstance(data, pd.Series):
        data = data.values
        
    def get_mdd(values): # values为np.array的净值曲线，初始资金为1
        dd = [values[i:].min() / values[i] - 1 for i in range(len(values))]
        return abs(min(dd))
    
    if not isinstance(data, pd.DataFrame):
        return get_mdd(data)
    else:
        return data.apply(get_mdd)
    
    
def cal_indicators(df_daily_return):
    '''
    功能：给定daily return，计算各组合的评价指标，包括：年化收益率、年化标准差、夏普值、最大回撤
    输入：
        df_daily_return  pd.DataFrame，index为升序排列的日期，columns为各组合名称，value为daily_return
    '''
    
    df_cum_value = (df_daily_return + 1).cumprod()
    res = pd.DataFrame(index=['年化收益率','年化标准差','夏普值','最大回撤'], columns=df_daily_return.columns, data=0.0)
    res.loc['年化收益率'] = (df_daily_return.mean() * 250).apply(lambda x: '%.2f%%' % (x*100))
    res.loc['年化标准差'] = (df_daily_return.std() * np.sqrt(250)).apply(lambda x: '%.2f%%' % (x*100))
    # sharpe ratio是不是少了一个
    res.loc['夏普值'] = (df_daily_return.mean() / df_daily_return.std() * np.sqrt(250)).apply(lambda x: np.round(x, 2))
    res.loc['最大回撤'] = cal_maxdrawdown(df_cum_value).apply(lambda x: '%.2f%%' % (x*100))
    return res


def cal_turnover(current_wts, target_wts):
    '''
   功能：给定当前持仓比例和目标持仓比例，计算双边换手率（默认持仓比例之和均为1）
   输入：
       current_wts，pd.Series，当前持仓比例
       target_wts，pd.Series，目标持仓比例
   输出：
       float
    '''
    if not isinstance(current_wts, pd.Series) or not isinstance(target_wts, pd.Series):
        raise ValueError('current_wts and target_wts should be pd.Series！！！')
    tmp = pd.merge(pd.DataFrame({'current_wts':current_wts}), pd.DataFrame({'target_wts':target_wts}), how='outer', left_index=True, right_index=True)
    tmp = tmp.fillna(0.0)
    tmp['turnover'] = tmp.current_wts - tmp.target_wts
    return abs(tmp['turnover']).sum()


def get_ticker_period_rtn(tickers, start_date, end_date):
    '''
    功能：输入tickers + 起始日期，获取tickers在这期间的daily return
    输入：
        ticker，list of ticker string
        start_date，yyyymmdd日期
        end_date，yyyymmdd日期
    输出：
        pd.DataFrame，index为日期yyyymmdd，columns为tickers string
    依赖：
        DataAPI：MktEqudAdjGet
        function：shift_date()
    '''
    begin_date = shift_date(start_date, 1)  # 向前推一天保证第一天也有daily return
    data = DataAPI.MktEqudAdjGet(ticker=tickers, beginDate=begin_date, endDate=end_date, field='ticker,tradeDate,closePrice', pandas='1')
    daily_rtn = data.pivot(index='tradeDate', columns='ticker', values='closePrice').pct_change().fillna(0.0)
    return daily_rtn